## Dependências

In [19]:
import plotly.express as px
import pandas as pd
import matplotlib.pyplot as plt

## Configurações

In [20]:
pd.options.plotting.backend = "plotly"

## Funções de apoio

In [21]:
def tratamento_data(data):
    return data[8:]+'/'+data[5:7]+'/'+data[0:4]

def determinar_situacao(obj):
    if isinstance(obj, pd.Series):
        obj['Situação'] = situacao[(obj['%EAR']>=situacao['min'])&(obj['%EAR']<situacao['max'])]['situação'].iloc[0]
    elif isinstance(obj, pd.DataFrame):
        for idx in obj.index:
            obj.loc[idx,'Situação'] = situacao[
                (obj.loc[idx,'%EAR']>=situacao['min'])&(obj.loc[idx,'%EAR']<situacao['max'])
            ]['situação'].iloc[0]

## Variáveis de referência

In [22]:
ear_columns_drop = ['cod_resplanejamento','nom_ree','id_subsistema','id_subsistema_jusante',
        'nom_subsistema_jusante','ear_reservatorio_subsistema_jusante_mwmes',
        'earmax_reservatorio_subsistema_jusante_mwmes']
new_columns_labels = ['Reservatório','Tipo','Bacia','Subsistema','Data','EAR','EARmax','%EAR']

## Entrada de dados

Aqui são lidos os dados de **EAR por bacia hidrográfica** e **EAR por reservatório** entre 2011 e 2021

In [23]:
ear_bacia = {}
ear_res = {}
ano_range = range(2011,2022)
for ano in ano_range:
    ear_bacia[ano] = pd.read_csv(f'EAR_Bacia/EAR_DIARIO_BACIAS_{ano}.csv',
                               sep=';')
for ano in ano_range:
    ear_res[ano] = pd.read_csv(f'EAR_Reservatorio/EAR_DIARIO_RESERVATORIOS_{ano}.csv',
                               sep=';')

## Tratamento coluna data

Aqui o formato de data **aaaa--mm-dd** é trocado pelo formato **dd/mm/aaaa**, para melhor entendimento

In [24]:
for ano in ano_range:
    for df in (ear_bacia[ano], ear_res[ano]):
        df['ear_data'] = df['ear_data'].map(tratamento_data)

## Classificador de situação

Aqui são criadas cinco faixas classificatórias da situação de energia armazenável para um determinado reservatório:
    - Entre 80% e 100%: Muito favorável
    - Entre 60% e 80%: Favorável
    - Entre 40% e 60%: Regular
    - Entre 20% e 40%: Desfavorável
    - Entre 0% e 20%: Crítico

In [25]:
situacao = pd.DataFrame(
    {
        'min':[80, 60, 40, 20, 0],
        'max':[100, 80, 60, 40, 20],
        'situação':['Muito favorável', 'Favorável', 'Regular', 'Desfavorável', 'Crítico']
    }
)

### Configuração da query

Aqui o usuário escolhe uma data-referência e um reservatório para a análise

In [26]:
data = '04/11/2021'
reservatorio = 'BALBINA'

## Situação do reservatório

Aqui, são mostrados os dados essenciais referentes ao reservatório solicitado na data-referência, com a respectiva situação da energia armazenável

In [27]:
ano = int(data[6:])
df = ear_res[ano]
res_query = df[
    (df['nom_reservatorio']==reservatorio)&(df['ear_data']==data)
    ].squeeze().drop(ear_columns_drop)
res_query.index = new_columns_labels
determinar_situacao(res_query)
res_query.name = res_query['Reservatório']
res_query

Reservatório                   BALBINA
Tipo            Reservatório com Usina
Bacia                         AMAZONAS
Subsistema                       NORTE
Data                        04/11/2021
EAR                         465.446014
EARmax                      776.494995
%EAR                         59.941898
Situação                       Regular
Name: BALBINA, dtype: object

## Situação dos reservatórios da mesma bacia

Aqui, são mostradas informações da bacia hidrográfica em que se situa o reservatório, para permitir uma análise comparativa a partir de uma tabela e gráficos

In [28]:
bacia = res_query['Bacia']
all_res_query = df[(df['nom_bacia']==bacia)&(df['ear_data']==data)
    ].drop(ear_columns_drop, axis=1)
all_res_query.columns = new_columns_labels
all_res_query['Situação'] = 0
determinar_situacao(all_res_query)
all_res_query.set_index('Reservatório', inplace=True)
all_res_query = all_res_query.round(2)
all_res_query

,Tipo,Bacia,Subsistema,Data,EAR,EARmax,%EAR,Situação
Reservatório,,,,,,,,
BALBINA,Reservatório com Usina,AMAZONAS,NORTE,04/11/2021,465.45,776.49,59.94,Regular
CURUA-UNA,Reservatório com Usina,AMAZONAS,NORTE,04/11/2021,22.38,27.27,82.06,Muito favorável
RONDON II,Reservatório com Usina,AMAZONAS,SUDESTE,04/11/2021,19.07,57.48,33.17,Desfavorável
SAMUEL,Reservatório com Usina,AMAZONAS,SUDESTE,04/11/2021,21.26,259.09,8.21,Crítico
SINOP,Reservatório com Usina,AMAZONAS,SUDESTE,04/11/2021,401.01,875.14,45.82,Regular


In [29]:
pie_bacia = all_res_query['EAR'].copy()
pie_bacia.name = ''
fig = px.pie(all_res_query, values='EAR', names=all_res_query.index, title='Energia Armazenável Absoluta')
fig.show()
bar = all_res_query['%EAR'].copy()
bar.plot(kind='bar', title='% Energia Armazenável')

## Situação das bacias do subsistema ao qual pertence o reservatório

Aqui, são mostradas as informações das bacias do subsistema ao qual pertence o reservatório, em forma de tabela e gráficos

In [30]:
subsistema = res_query['Subsistema']
subsistema_query = df[(df['nom_subsistema']==subsistema)&(df['ear_data']==data)]
bacias = set(subsistema_query['nom_bacia'])
subsistema_data = ear_bacia[ano][(ear_bacia[ano]['nomecurto'].isin(bacias))&(ear_bacia[ano]['ear_data']==data)].copy()
subsistema_data.columns = ['Bacia', 'Data', 'EARmax', 'EAR', '%EAR']
subsistema_data.set_index('Bacia', inplace=True)
subsistema_data = pd.DataFrame({'Data':subsistema_data['Data'],
                           'EAR':subsistema_data['EAR'],
                           'EARmax':subsistema_data['EARmax'],
                           '%EAR':subsistema_data['%EAR']})
determinar_situacao(subsistema_data)
print('Subsistema ' + subsistema)
subsistema_data

Subsistema NORTE


,Data,EAR,EARmax,%EAR,Situação
Bacia,,,,,
AMAZONAS,04/11/2021,929.163025,1995.489014,46.563202,Regular
TOCANTINS,04/11/2021,13546.129883,49268.960938,27.494200,Desfavorável


In [31]:
fig = px.pie(subsistema_data, values='EAR', names=subsistema_data.index, title='Energia Armazenável Absoluta')
fig.show()
bar = subsistema_data['%EAR'].copy()
bar.plot(kind='bar', title='% Energia Armazenável')

## Histórico do reservatório

Aqui, um histórico de energia armazenável média mensal do reservatório é levantado, tomando como base o intervalo 2011-2021. A finalidade é mostrar para o usuário quais os períodos históricos mais favoráveis e menos favoráveis para a geração de energia elétrica

In [32]:
def mes_to_str(mes):
    if mes<10:
        return '0' + str(mes)
    else:
        return str(mes)

num_meses = range(1,13)
nomes_meses = ['Jan','Fev','Mar','Abr','Mai','Jun','Jul','Ago','Set','Out','Nov','Dez']
tradutor_nummes_nomemes = pd.Series(data=nomes_meses, index=num_meses)

In [33]:
historico = pd.DataFrame(
    columns=nomes_meses,
    index=ano_range,
    dtype=float
)

In [34]:
for ano in ano_range:
    for mes in range(1,13):
        df = ear_res[ano]
        serie = df[
            (df['ear_data'].str[3:5]==mes_to_str(mes))&(df['nom_reservatorio']==reservatorio)
            ]['ear_reservatorio_subsistema_proprio_mwmes']
        historico.loc[ano, tradutor_nummes_nomemes[mes]] = serie.mean()

stats = historico.describe().round(2).transpose()

## Média de EAR por mês ao longo dos últimos 10 anos

In [35]:
stats['mean'].plot()